## Defining the linear progam

Decision Variables:
- $x_{i}$ represents allocation of portfolio to asset $i$.

Contraints:
- $\sum_{i} x_i = 1$
- $x_i \ge 0 \forall i$

Objective function:
$$ \max_x \sum_{j} p_j \times (x^T \mu_{j} - \lambda \times x^T \Sigma_j x)$$

Constants being used:
- $\mu_j$, $\Sigma_j$ are expected returns and covariance matrix under scenario $j$
- $p_j$ is the probability of being in scenario $j$
- $\lambda$ is a constant representing risk tolerance.

In [62]:
using JuMP, Gurobi, LinearAlgebra, CSV

In [63]:
m = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [64]:
num_assets = 4

@variable(m, x[1:num_assets])

@constraint(m, non_neg[i= 1:num_assets], x[i] >= 0)
@constraint(m, sum(x[i] for i=1:num_assets) == 1)

x[1] + x[2] + x[3] + x[4] == 1.0

In [65]:
CSV.read("data/Weak_ExpReturns.csv", header=0)

,Column1,Column2
,String,Float64
1,SPY_Return,0.108971
2,US_Corp_Return,0.0833257
3,TreasInd_Return,0.0071563
4,Cash_Return,0.000886495


In [66]:
#Have to load the data.
weak_mu = CSV.read("data/Weak_ExpReturns.csv", header=0)[:, 2]
weak_sigma = convert(Matrix, CSV.read("data/Weak_CovMat.csv", header=1)[:, 2:num_assets+1])

normal_mu = CSV.read("data/Normal_ExpReturns.csv", header=0)[:, 2]
normal_sigma = convert(Matrix, CSV.read("data/Normal_CovMat.csv", header=1)[:, 2:num_assets+1])

strong_mu = CSV.read("data/Strong_ExpReturns.csv", header = 0)[:, 2]
strong_sigma = convert(Matrix, CSV.read("data/Strong_CovMat.csv", header=1)[:, 2:num_assets+1])

4×4 Array{Float64,2}:
  0.190415      0.0071689   -0.000709067  -5.29306e-6
  0.0071689     0.0163268    0.00117289   -5.6025e-6 
 -0.000709067   0.00117289   0.000620389  -3.23118e-6
 -5.29306e-6   -5.6025e-6   -3.23118e-6    6.96783e-7

In [67]:
probs = CSV.read("data/theta0_Probs.csv")[1, 2:num_assets]
normal_prob = probs[1]

strong_prob = probs[2]

weak_prob = probs[3]

0.2797195166414282

In [68]:
transpose(x)*weak_mu

0.10897103662586333 x[1] + 0.08332572891330764 x[2] + 0.0071562978966418596 x[3] + 0.000886494873529021 x[4]

In [87]:
lambda = 4
@objective(m, Max, weak_prob*(transpose(x)*weak_mu - lambda * transpose(x)*weak_sigma*x) + normal_prob*(transpose(x)*normal_mu - lambda * transpose(x)*normal_sigma*x) + strong_prob*(transpose(x)*strong_mu - lambda*transpose(x)*strong_sigma*strong_mu))

-0.9092075932615357 x[1]² - 0.026398714952623192 x[1]*x[2] + 0.010833817879402633 x[1]*x[3] + 0.00035546235048013945 x[1]*x[4] - 0.0710274283285197 x[2]² - 0.005646710591803868 x[2]*x[3] - 0.00015319631401537282 x[2]*x[4] - 0.002327012649860939 x[3]² - 1.635854307563336e-5 x[3]*x[4] - 2.8645121809920462e-6 x[4]² + 0.1273568828395262 x[1] + 0.06919209014477558 x[2] + 0.001773428176372963 x[3] + 0.0008409759329278784 x[4]

In [88]:
optimize!(m)

Optimize a model with 5 rows, 4 columns and 8 nonzeros
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 1e-01]
  QObjective range [6e-06, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 4 columns, 4 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 6.000e+00
 Factor NZ  : 1.000e+01
 Factor Ops : 3.000e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.61366535e+05  2.61637503e+05  3.50e+03 1.27e-01  1.00e+06     0s
   1  -1.20666657e+04  1.25991493e+04  7.29e+01 1.16e-08  2.68e+04     0s
   2   3.43766218e-02  7.39801608e+02  5.22e-01 8.32e-11  3.78e+02     0s
   3   1.88829713e-02  2.04539882

In [89]:
#SPY, US_Corp, TreasIndex, Cash
value.(x)

4-element Array{Float64,1}:
 0.06288141730091003  
 0.469337579373579    
 1.7316204457363596e-6
 0.4677792717050652   